# 1.3 Tagging, using function calling / tool use

(Adapted from https://python.langchain.com/docs/use_cases/tagging) 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/tagging.ipynb)

## Use case

Tagging means labeling a document with classes such as:

- sentiment
- language
- style (formal, informal etc.)
- covered topics
- political tendency

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/tagging.png?raw=1)

## Overview

Tagging has a few components:

* `function`: Like [extraction](/docs/use_cases/extraction), tagging uses [functions](https://openai.com/blog/function-calling-and-other-api-updates) to specify how the model should tag a document
* `schema`: defines how we want to tag the document

## Quickstart

Let's see a very straightforward example of how we can use OpenAI tool calling for tagging in LangChain. We'll use the [`with_structured_output`](/docs/modules/model_io/chat/structured_output) method supported by OpenAI models:

In [28]:
%pip install python-dotenv langchain langchain-openai docarray --upgrade --quiet
%pip install pydantic==1.10.10 --upgrade --quiet

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

Let's specify a Pydantic model with a few properties and their expected type in our schema.

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM

# tagging_functions = [convert_to_openai_function(Classification)]
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").bind(functions=tagging_functions,
#                                                                  function_call={"name": "Classification"})

#tagging_chain = tagging_prompt | llm | JsonOutputFunctionsParser()

# If we only want to extract one field, we can use the JsonKeyOutputFunctionsParser
# tagging_chain = tagging_prompt | llm | JsonKeyOutputFunctionsParser(key_name="language")

# Newer syntax, currently in Beta:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [30]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

If we want JSON output, we can just call `.dict()`

In [31]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": inp})
res.dict()

{'sentiment': 'negative', 'aggressiveness': 8, 'language': 'Spanish'}

As we can see in the examples, it correctly interprets what we want.

The results vary so that we may get, for example, sentiments in different languages ('positive', 'enojado' etc.).

We will see how to control these results in the next section.

## Finer control

Careful schema definition gives us more control over the model's output.

Specifically, we can define:

- possible values for each property
- description to make sure that the model understands the property
- required properties to be returned

Let's redeclare our Pydantic model to control for each of the previously mentioned aspects using enums:

In [32]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [33]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

chain = tagging_prompt | llm

Now the answers will be restricted in a way we expect!

In [34]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.invoke({"input": inp})

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [35]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.invoke({"input": inp})

Classification(sentiment='sad', aggressiveness=5, language='spanish')

In [36]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.invoke({"input": inp})

Classification(sentiment='neutral', aggressiveness=2, language='english')

The [LangSmith trace](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r) lets us peek under the hood:

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/tagging_trace.png?raw=1)

### Going deeper

* You can use the [metadata tagger](/docs/integrations/document_transformers/openai_metadata_tagger) document transformer to extract metadata from a LangChain `Document`.
* This covers the same basic functionality as the tagging chain, only applied to a LangChain `Document`.